In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip

--2021-02-04 08:01:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116104947 (111M) [application/x-httpd-php]
Saving to: ‘data.zip’

data.zip            100%[===================>] 110.73M  38.1MB/s    in 2.9s    

2021-02-04 08:01:44 (38.1 MB/s) - ‘data.zip’ saved [116104947/116104947]



In [2]:
!unzip data.zip -d data

Archive:  data.zip
  inflating: data/BitcoinHeistData.csv  


In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gzip, pickle


In [55]:
df=pd.read_csv('/content/data/BitcoinHeistData.csv')

In [56]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [57]:
df.loc[0]

address      111K8kZAEnJg245r2cM6y9zgJGHZtJPy6
year                                      2017
day                                         11
length                                      18
weight                              0.00833333
count                                        1
looped                                       0
neighbors                                    2
income                              1.0005e+08
label                          princetonCerber
Name: 0, dtype: object

In [58]:
lb=LabelEncoder()
df['address']=lb.fit_transform(df['address'])
df['label']=lb.fit_transform(df['label'])

In [59]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,23,2017,11,18,0.008333,1,0,2,100050000.0,26
1,128,2016,132,44,0.000244,1,0,1,100000000.0,27
2,169,2016,246,0,1.000000,1,0,2,200000000.0,26
3,217,2016,322,72,0.003906,1,0,2,71200000.0,26
4,293,2016,238,144,0.072848,456,0,1,200000000.0,27


In [60]:
new_df=df.sample(frac=1)

In [65]:
new_df.loc[new_df['address'] == 128]

,address,year,day,length,weight,count,looped,neighbors,income,label
1,128,2016,132,44,0.000244,1,0,1,100000000.0,27


In [77]:
new_df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
2285858,808373,2017,65,28,0.001953,1,0,2,273035227.0,28
1689894,1710014,2015,199,12,0.041667,1,0,2,37081927.0,28
1158069,1531103,2014,32,0,1.000000,1,0,2,62731414.0,28
2705260,944344,2018,119,0,0.076923,1,0,2,150000000.0,28
2361824,1326650,2017,141,6,0.125000,1,0,2,98834702.0,28


In [73]:
Y=new_df['label']
X=new_df.drop('label',axis=1)

In [74]:
X.head()

,address,year,day,length,weight,count,looped,neighbors,income
2285858,808373,2017,65,28,0.001953,1,0,2,273035227.0
1689894,1710014,2015,199,12,0.041667,1,0,2,37081927.0
1158069,1531103,2014,32,0,1.000000,1,0,2,62731414.0
2705260,944344,2018,119,0,0.076923,1,0,2,150000000.0
2361824,1326650,2017,141,6,0.125000,1,0,2,98834702.0


In [76]:
Y.head()

2285858    28
1689894    28
1158069    28
2705260    28
2361824    28
Name: label, dtype: int64

In [78]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.5,shuffle=False)

In [79]:
from joblib import Parallel, delayed,dump
# from sklearn.externals import joblib 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle

In [80]:
model_rf=RandomForestClassifier(max_depth=2,random_state=42)
model_dt=DecisionTreeClassifier(splitter="random",max_depth=2,random_state=2,max_features="auto")
models=[]
models.append(model_rf)
models.append(model_dt)

TRAINING MODEL AND SAVING MODEL IN .PKL FILE

In [127]:


def train_model(X,y,model,count):
  print('Executing model number',count)
  output=model.fit(X,y)
  # filename='./'+str(count)+'.pkl'
  with gzip.open('./'+str(count)+'.pkl', 'wb') as ofp:
    pickle.dump(output, ofp)
  return output




In [128]:
%%time

result = Parallel(n_jobs=2)(delayed(train_model)(X_train, y_train, models[i],i) for i in range(len(models)))

CPU times: user 158 ms, sys: 623 ms, total: 781 ms
Wall time: 1min 35s


LOADING MODEL IN PARALLLEL AND MAKING PREDICTION ON TEST SET

In [153]:
def test_model(X_test,y_test,model,model_count):

  filename='/content/'+str(model_count)+'.pkl'
  with gzip.open(filename, 'rb') as ifp:
    loaded_model=pickle.load(ifp)
    res=loaded_model.predict(X_test)
  return res


In [155]:
%%time

result = Parallel(n_jobs=2)(delayed(test_model)(X_test, y_test, result[i],i) for i in range(len(models)))

CPU times: user 112 ms, sys: 170 ms, total: 283 ms
Wall time: 25.9 s


In [166]:
result_df=pd.DataFrame({'RF':result[0],'DT':result[1]})

In [171]:
result_df['Average']=result_df.mean(axis=1)

In [172]:
result_df 

,RF,DT,Average
0,28,28,28.0
1,28,28,28.0
2,28,28,28.0
3,28,28,28.0
4,28,28,28.0
...,...,...,...
1458344,28,28,28.0
1458345,28,28,28.0
1458346,28,28,28.0
1458347,28,28,28.0
